- Comisiones = 0.125%
- No hay Impuestos
- Margin Account = 50% 
- $ 1,000,000 USD Cash
- N Shares = 50
- Stop Loss, Take Profit = 6%
- Buy/Sell at close
- Borrow Rate = 0.25%

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ta
from dataclasses import dataclass

sns.set_theme()

In [16]:
data = pd.read_csv('Data/aapl_5m_train.csv')
data.head()

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume
0,0,1609770600,0,2021-01-04 14:30:00,133.570007,133.611602,132.389999,132.809997,6624663.0
1,1,1609770900,0,2021-01-04 14:35:00,132.750000,132.750000,131.809997,131.889999,2541553.0
2,2,1609771200,0,2021-01-04 14:40:00,131.500000,132.339996,131.500000,132.059997,2492415.0
3,3,1609771500,0,2021-01-04 14:45:00,132.000000,132.250000,131.899993,132.250000,1859131.0
4,4,1609771800,0,2021-01-04 14:50:00,132.000000,132.018096,131.520004,131.589996,1780105.0


In [17]:
rsi_indicator = ta.momentum.RSIIndicator(data.Close, window=15)

data['rsi'] = rsi_indicator.rsi()
data['buy_signal'] = data['rsi'] < 16
data['sell_signal'] = data['rsi'] > 80

data = data.dropna()


In [18]:
COM = 0.125 / 100
BORROR_RATE = 0.25 / 100

# DOF
STOP_LOSS = 0.1
TAKE_PROFIT = 0.1
N_SHARES = 500

capital = 1_000_000
active_long_positions = []

In [19]:
@dataclass
class Position:
     """
     Data class for a trading position
     """
     ticker: str
     n_shares: int
     price: float
     sl: float
     tp: float
     time: str

In [20]:
for i, row in data.iterrows():
     # --- LONG --- #
     # Check active orders
     for position in active_long_positions.copy():
          # Stop loss or take profit check
          if row.Close > position.tp or row.Close < position.sl:
               # Add profit / losses to capital
               capital += row.Close * position.n_shares * (1 - COM)
               # Remove position from active pos
               active_long_positions.remove(position)

     # Check Signal
     if row.buy_signal:
          cost = row.Close * N_SHARES * (1 + COM)
          # Do we have enough cash?
          if capital > cost:
               # Discount cash
               capital -= cost
               # Add position to portfolio
               pos = Position(ticker = 'AAPL', n_shares = N_SHARES, price = row.Close, 
                              sl = row.Close * (1 - STOP_LOSS), tp = row.Close * (1 + TAKE_PROFIT),
                              time = row.Datetime)
               active_long_positions.append(pos)

# At the end of the backtesting, we should close all active positions
capital += row.Close * len(active_long_positions) * N_SHARES * (1 - COM)
active_long_positions = []

print(f'${capital:,.2f}')

$1,358,884.47
